## Torchreid getting started
- reference : https://kaiyangzhou.github.io/deep-person-reid/

터미널에서 다음 명령을 실행한 후 새롭게 생성된 `torchreid` 커널로 본 노트북을 실행합니다.

```sh
# cd to your preferred directory and clone this repo
git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
cd deep-person-reid/
conda create --name torchreid python=3.7
conda activate torchreid

# install dependencies
# make sure `which python` and `which pip` point to the correct path
pip install -r requirements.txt

# On cpu instance, install torch and torchvision (select the proper cuda version to suit your machine)
conda install pytorch-cpu torchvision-cpu -c pytorch

# On gpu instance, install torch and torchvision (select the proper cuda version to suit your machine)
conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
python setup.py develop

conda install jupyter
python -m ipykernel install --name torchreid --user
```

#### Step 1: import the Torchreid library

In [1]:
import torchreid

#### Step 2: construct data manager
아래코드는 자동으로 market-1501 데이터를 다운로드합니다.  
만약 market-1501 데이터가 다운되지 않을 경우 Kaggle(https://www.kaggle.com/datasets/pengcw1/market-1501) 에서 데이터를 다운받고 다음 경로로 업로드한 후 다음 셀을 다시 실행합니다.

```sh
./reid-data/market1501
```

다른 데이터셋을 이용하고자 할 경우 다음 페이지를 참고합니다. -> https://kaiyangzhou.github.io/deep-person-reid/datasets.html

In [2]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


/home/ec2-user/SageMaker/deep-person-reid/torchreid/data/datasets/image/market1501.py:38: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  'The current data structure is deprecated. Please '


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




#### Step 3: Construct CNN model and initialise optimiser/learning rate scheduler

`resnet50`를 사용하는 pretrained 모델을 사용하도록 선언합니다.

In [3]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

# model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

#### Step 4: construct engine

In [4]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

#### Step 5: run model training and test

테스트 런을 실행합니다.
실행시 사용할 수 있는 파라미터는 다음과 같습니다.
- save_dir (str): directory to save model.
- max_epoch (int): maximum epoch.
- start_epoch (int, optional): starting epoch. Default is 0.
- print_freq (int, optional): print_frequency. Default is 10.
- fixbase_epoch (int, optional): number of epochs to train ``open_layers`` (new layers)
    while keeping base layers frozen. Default is 0. ``fixbase_epoch`` is counted
    in ``max_epoch``.
- open_layers (str or list, optional): layers (attribute names) open for training.
- start_eval (int, optional): from which epoch to start evaluation. Default is 0.
- eval_freq (int, optional): evaluation frequency. Default is -1 (meaning evaluation
    is only performed at the end of training).
- test_only (bool, optional): if True, only runs evaluation on test datasets.
    Default is False.
- dist_metric (str, optional): distance metric used to compute distance matrix
    between query and gallery. Default is "euclidean".
- normalize_feature (bool, optional): performs L2 normalization on feature vectors before
    computing feature distance. Default is False.
- visrank (bool, optional): visualizes ranked results. Default is False. It is recommended to
    enable ``visrank`` when ``test_only`` is True. The ranked images will be saved to
    "save_dir/visrank_dataset", e.g. "save_dir/visrank_market1501".
- visrank_topk (int, optional): top-k ranked images to be visualized. Default is 10.
- use_metric_cuhk03 (bool, optional): use single-gallery-shot setting for cuhk03.
    Default is False. This should be enabled when using cuhk03 classic split.
- ranks (list, optional): cmc ranks to be computed. Default is [1, 5, 10, 20].
- rerank (bool, optional): uses person re-ranking (by Zhong et al. CVPR'17).
    Default is False. This is only enabled when test_only=True.

In [8]:
%%time
engine.run(
    save_dir="log/resnet50",
    # max_epoch=60,
    # eval_freq=10,
    # print_freq=10,
    test_only=True,
    visrank=True,
    # visrank_topk=20
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 5.1423 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 2.4%
CMC curve
Rank-1  : 8.1%
Rank-5  : 16.4%
Rank-10 : 22.5%
Rank-20 : 29.3%
# query: 3368
# gallery 15913
Visualizing top-20 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3